In [3]:
import functionlib as fl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
def payoff_barrier_option(path, K, T, B, option_type, observ_freq=1, graph=False):
    """
    Computes the payoff of a barrier option given a price path. 
    Option type can be: upincall, upoutcall, upinput, upoutput, downincall, downoutcall, downinput, downoutput
    Observation are made every observ_freq days.
    """
    
    #if path.shape[1] > 1:
        #raise ValueError("Path contains more than 1 simulation")



    if graph:
        plt.figure(figsize=(10,6))
        plt.plot(path)
        plt.axhline(y = B, color='r', linestyle='--')
        plt.text(0, B+0.2, 'BARRIER', color='r')
        plt.xlabel('Time (days)')
        plt.ylabel('Index level')
        plt.grid()
        plt.show()



    time_steps = T*252
    observation_dates = np.arange(observ_freq, time_steps+1, observ_freq)

    path_at_observation = [path.iloc[observation-1].values for observation in observation_dates]
    
    max_price = max(path_at_observation)
    min_price = min(path_at_observation)

    S_T = path.iloc[-1].values[0]

    payoff = 0

    if option_type == 'upincall':
        if max_price >= B: #Barrier reached, standard call payoff
            return max(S_T-K, 0)
        else:
            return 0
        
    elif option_type == 'upinput': #Barrier reached, standard put payoff
        if max_price >= B:
            return max(K-S_T,0)
        else:
            return 0
    
    elif option_type == 'downincall':
        if min_price <= B: #Barrier reached, standard call payoff
            return max(S_T-K,0)
        else:
            return 0

    elif option_type == 'downinput':
        if min_price <= B:
            return max(K-S_T,0)
        else:
            return 0
        
    """
    All Knock-In cases are treated, to compute the payoff for Knock-Out options,
    we use parity relationhip: KO + KI = Vanilla
    """

    if option_type == 'upoutcall':
        return max(S_T-K, 0) - payoff_barrier_option(path, K, T, B, option_type='upincall', observ_freq=observ_freq)
    
    elif option_type == 'upoutput':
        return max(K-S_T, 0) - payoff_barrier_option(path, K, T, B, option_type='upinput', observ_freq=observ_freq)
    
    elif option_type == 'downoutcall':
        return max(S_T-K, 0) - payoff_barrier_option(path, K, T, B, option_type='downincall', observ_freq=observ_freq)
    
    else:
        return max(K-S_T, 0) - payoff_barrier_option(path, K, T, B, option_type='downinput', observ_freq=observ_freq)
    



def barrier_option_price(S0, K, T, r, vol, B, option_type, observ_freq=1, M=1000):

    barrier_payoff = lambda path: payoff_barrier_option(path, K, T, B, option_type, observ_freq)

    path_matrix = fl.monte_carlo(S0, T, r, vol, M, antithetic=False)
    payoff_simulation = np.array([barrier_payoff(path_matrix.iloc[:,i].to_frame()) for i in range(path_matrix.shape[1])])

    return np.exp(-r*T)*payoff_simulation.mean()
    


In [ ]:
from scipy.stats import norm

def upincallprice(S0, K, T, r, vol, B):
    """Closed formula for an up and in call price (continuously monitored)"""

    lamb = (r + vol**2/2)/vol**2
    x1 = np.log(S0/B)/(vol*np.sqrt(T)) + lamb*vol*np.sqrt(T)
    y1 = np.log(B/S0)/(vol*np.sqrt(T)) + lamb*vol*np.sqrt(T)
    y = np.log(B**2/(S0*K))/(vol*np.sqrt(T)) + lamb*vol*np.sqrt(T)

    return S0*norm.cdf(x1, 0, 1) - K*np.exp(-r*T)*norm.cdf(x1 - vol*np.sqrt(T), 0, 1) - S0*(B/S0)**(2*lamb)*(norm.cdf(-y, 0, 1) - norm.cdf(-y1, 0, 1)) + K*np.exp(-r*T)*(B/S0)**(2*lamb-2)*(norm.cdf(-y + vol*np.sqrt(T), 0, 1) - norm.cdf(-y1 + vol*np.sqrt(T), 0, 1))


6.940981136940344

In [14]:
S0 = 100
K = 100
T = 1
r = 0.02
vol = 0.15
B = 105
option_type = 'upincall'

print(f'Price with Monte-Carlo simulation: {barrier_option_price(S0, K, T, r, vol, B, option_type)}')
print(f'Price with closed formula: {upincallprice(S0, K, T, r, vol, B)}')

Price with Monte-Carlo simulation: 7.195650493922509
Price with closed formula: 6.940981136940344
